# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 20 2022 1:04PM,246842,10,MSP214126,"Methapharm, Inc.",Released
1,Sep 20 2022 1:04PM,246842,10,MSP214133,"Methapharm, Inc.",Released
2,Sep 20 2022 1:04PM,246842,10,MSP214132,"Methapharm, Inc.",Released
3,Sep 20 2022 1:04PM,246842,10,MSP214141,"Methapharm, Inc.",Released
4,Sep 20 2022 12:57PM,246840,10,MSP214103,"Methapharm, Inc.",Released
5,Sep 20 2022 12:57PM,246840,10,MSP214117,"Methapharm, Inc.",Released
6,Sep 20 2022 12:57PM,246840,10,MSP214120,"Methapharm, Inc.",Released
7,Sep 20 2022 12:57PM,246840,10,MSP214121,"Methapharm, Inc.",Released
8,Sep 20 2022 12:57PM,246840,10,MSP214127,"Methapharm, Inc.",Released
9,Sep 20 2022 12:57PM,246840,10,MSP214128,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,246835,Executing,4
28,246835,Released,18
29,246836,Released,1
30,246840,Released,12
31,246842,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246834,NaN,NaN,1.0
246835,NaN,4.0,18.0
246836,NaN,NaN,1.0
246840,NaN,NaN,12.0
246842,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,0.0,9.0,23.0
246762,0.0,0.0,1.0
246767,0.0,1.0,0.0
246770,0.0,1.0,0.0
246785,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,0,9,23
246762,0,0,1
246767,0,1,0
246770,0,1,0
246785,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,0,9,23
1,246762,0,0,1
2,246767,0,1,0
3,246770,0,1,0
4,246785,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,,9,23
1,246762,,,1
2,246767,,1,
3,246770,,1,
4,246785,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 20 2022 1:04PM,246842,10,"Methapharm, Inc."
4,Sep 20 2022 12:57PM,246840,10,"Methapharm, Inc."
16,Sep 20 2022 12:36PM,246836,10,Cosmetic Solutions LLC
17,Sep 20 2022 12:35PM,246835,19,"AdvaGen Pharma, Ltd"
39,Sep 20 2022 12:31PM,246834,16,Sartorius Stedim North America
40,Sep 20 2022 12:28PM,246832,16,Sartorius Stedim North America
41,Sep 20 2022 12:28PM,246828,20,"ACI Healthcare USA, Inc."
118,Sep 20 2022 12:25PM,246831,16,Sartorius Stedim North America
119,Sep 20 2022 12:24PM,246830,16,Sartorius Stedim North America
120,Sep 20 2022 12:06PM,246827,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 20 2022 1:04PM,246842,10,"Methapharm, Inc.",,,4
1,Sep 20 2022 12:57PM,246840,10,"Methapharm, Inc.",,,12
2,Sep 20 2022 12:36PM,246836,10,Cosmetic Solutions LLC,,,1
3,Sep 20 2022 12:35PM,246835,19,"AdvaGen Pharma, Ltd",,4,18
4,Sep 20 2022 12:31PM,246834,16,Sartorius Stedim North America,,,1
5,Sep 20 2022 12:28PM,246832,16,Sartorius Stedim North America,,,1
6,Sep 20 2022 12:28PM,246828,20,"ACI Healthcare USA, Inc.",,,77
7,Sep 20 2022 12:25PM,246831,16,Sartorius Stedim North America,,,1
8,Sep 20 2022 12:24PM,246830,16,Sartorius Stedim North America,,,1
9,Sep 20 2022 12:06PM,246827,10,Eywa Pharma Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 1:04PM,246842,10,"Methapharm, Inc.",4,,
1,Sep 20 2022 12:57PM,246840,10,"Methapharm, Inc.",12,,
2,Sep 20 2022 12:36PM,246836,10,Cosmetic Solutions LLC,1,,
3,Sep 20 2022 12:35PM,246835,19,"AdvaGen Pharma, Ltd",18,4,
4,Sep 20 2022 12:31PM,246834,16,Sartorius Stedim North America,1,,
5,Sep 20 2022 12:28PM,246832,16,Sartorius Stedim North America,1,,
6,Sep 20 2022 12:28PM,246828,20,"ACI Healthcare USA, Inc.",77,,
7,Sep 20 2022 12:25PM,246831,16,Sartorius Stedim North America,1,,
8,Sep 20 2022 12:24PM,246830,16,Sartorius Stedim North America,1,,
9,Sep 20 2022 12:06PM,246827,10,Eywa Pharma Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 1:04PM,246842,10,"Methapharm, Inc.",4,,
1,Sep 20 2022 12:57PM,246840,10,"Methapharm, Inc.",12,,
2,Sep 20 2022 12:36PM,246836,10,Cosmetic Solutions LLC,1,,
3,Sep 20 2022 12:35PM,246835,19,"AdvaGen Pharma, Ltd",18,4,
4,Sep 20 2022 12:31PM,246834,16,Sartorius Stedim North America,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 1:04PM,246842,10,"Methapharm, Inc.",4.0,NaN,NaN
1,Sep 20 2022 12:57PM,246840,10,"Methapharm, Inc.",12.0,NaN,NaN
2,Sep 20 2022 12:36PM,246836,10,Cosmetic Solutions LLC,1.0,NaN,NaN
3,Sep 20 2022 12:35PM,246835,19,"AdvaGen Pharma, Ltd",18.0,4.0,NaN
4,Sep 20 2022 12:31PM,246834,16,Sartorius Stedim North America,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 20 2022 1:04PM,246842,10,"Methapharm, Inc.",4.0,0.0,0.0
1,Sep 20 2022 12:57PM,246840,10,"Methapharm, Inc.",12.0,0.0,0.0
2,Sep 20 2022 12:36PM,246836,10,Cosmetic Solutions LLC,1.0,0.0,0.0
3,Sep 20 2022 12:35PM,246835,19,"AdvaGen Pharma, Ltd",18.0,4.0,0.0
4,Sep 20 2022 12:31PM,246834,16,Sartorius Stedim North America,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2468221,36.0,8.0,0.0
15,740334,56.0,51.0,0.0
16,1480883,5.0,1.0,0.0
19,246835,18.0,4.0,0.0
20,987224,89.0,6.0,8.0
21,246770,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2468221,36.0,8.0,0.0
1,15,740334,56.0,51.0,0.0
2,16,1480883,5.0,1.0,0.0
3,19,246835,18.0,4.0,0.0
4,20,987224,89.0,6.0,8.0
5,21,246770,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,36.0,8.0,0.0
1,15,56.0,51.0,0.0
2,16,5.0,1.0,0.0
3,19,18.0,4.0,0.0
4,20,89.0,6.0,8.0
5,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,36.0
1,15,Released,56.0
2,16,Released,5.0
3,19,Released,18.0
4,20,Released,89.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,8.0,0.0
Executing,8.0,51.0,1.0,4.0,6.0,1.0
Released,36.0,56.0,5.0,18.0,89.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,8.0,0.0
1,Executing,8.0,51.0,1.0,4.0,6.0,1.0
2,Released,36.0,56.0,5.0,18.0,89.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,8.0,0.0
1,Executing,8.0,51.0,1.0,4.0,6.0,1.0
2,Released,36.0,56.0,5.0,18.0,89.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()